# 앙상블

- 여러 머신러닝 모델을 결합하여 더 좋은 모델을 얻는 방법
- 앙상블의 종류
    - 보팅
    - 배깅
    - 부스팅 

## 보팅(Voting)

<img src = "./image/voting.png">

- 오른쪽 그림이 소프트 보팅 

- 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정
- 종류 
    - 하드 보팅(Hard Voting)
        - 다수의 분류기가 예측한 결과값을 최종 결과로 결정
        
    - 소프트 보팅(Soft Voㅇting)
        - 모든 분류기가 예측한 결정 확률의 평균이 가장 높은 결과값을 최종 결과로 선정 

## 배깅 (Bagging)

<img src = "./image/bagging.png">

- Decision Tree를 사용하면 다 똑같은 알고리즘으로 해결해야 함 

- 데이터 샘플링(Bootstrap)을 통해 모델을 학습시키고 결과를 집계하는 방법
    - Bootstrap
        - 데이터가 조금씩은 편향되도록 샘플링하는 기법
        - 분산이 높은 모델의 **과대적합 위험을 줄이는 효과**가 있음
        
- 모두 같은 유형의 알고리즘 기반의 분류기를 사용 
- 데이터 분할 시 중복을 허용
- 예시 : 랜덤포레스트(RandomForest) - 실무에서는 랜덤포레스트를 많이쓰고 decision tree 많이 안씀
    - 과대 적합되기 쉬운 의사결정나무의 과대적합을 줄여 성능을 높일 수 있음 

- 배깅 방법은 일반적으로 높은 분산과 낮은 편향을 보이는 약한 학습기에 사용되는 반면, 부스팅 방법은 낮은 분산과 높은 편향이 관찰되는 경우에 활용됩니다.

## 부스팅(Boosting)

<img src = "./image/boosting.png">

- 여러 개의분류기가 **순차적**으로 학습을 수행 
- 이전 분류기의 예측이 틀린 데이터에 대해서 올바르게 예측할 수 있도록 **다음 분류기에 가중치(weight)** 를 부여하면서 학습과 예측을 진행
- 계속하여 분류기에게 가중치를 부스팅하며 학습을 진행하기 때문에 부스팅 방식이라고 불림 
- 일반적으로 부스팅 방식이 배깅에 비해 **성능이 좋지만 속도가 느리고 과적합이 발생할 가능성이** 더 높음
- 예시 : XGBoost, LightGBM

- 의사 결정나무는 성능이 그냥 그렇지만 속도가 빠름
- 적절한 앙상블을 써야 함 

- 프젝 순서 :
    - 데이터 수집, 전처리 + 분석, 모델링, 앙상블 

# 랜덤포레스트 

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                               GradientBoostingClassifier)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('./data/wine.csv')

In [4]:
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
x = df[['alcohol', 'sugar', 'pH']]
y = df['class']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y,
                                                    random_state = 11)

In [7]:
# 랜덤포레스트에 하이퍼파리미터를 따져보자 
# n: 빈도, 갯수 
# estimator : 모델들 , 즉 n_estimator : 모델 갯수 
# criterion = gini : 지니 불순도 

# 사이킷런의 랜덤포레스트는 기본적으로 100개의 결정 트리를 사용함 
rf = RandomForestClassifier(n_jobs = -1, random_state = 11)

In [8]:
scores = cross_validate(rf, x_train, y_train, return_train_score = True, n_jobs = -1)

In [9]:
# return_train_score : 교차 검증 시 훈련 세트에 대한 점수도 함께 변환
scores = cross_validate(rf, x_train, y_train, return_train_score = True, n_jobs = -1)

In [10]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9980276897524913 0.8905123269415858


- 랜덤포레스트는 의사결정나무의 앙상블이기 때문에 의사결정나무에서 제공하는 주요 매개변수는 모두 제공함
- 특성 중요도 계산도 가능 
    - 랜덤포레스트의 특성중요도는 각 의사결정나무의 특성 중요도를 취합 

In [11]:
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=11)

In [12]:
# 알콜, 슈가, pH 
# 특성 중요도를 개선해냄 
rf.feature_importances_

array([0.24026188, 0.47900383, 0.28073429])

- 의사결정나무에 비해 두번째 특성인 sugar의 중요도가 감소하고 alcohol과 pH의 중요도가 상승함 
    - 랜덤포레스트는 하나의 특성에 과도하게 집중되지 않고 더 많은 특성이 훈련에 기여할 기회를 얻음 
    - 과대적합이 줄어들고 일반화 성능을 높일 수 있음 

In [13]:
# 부트스트랩 때문에 생기는 또 하나의 특성
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 11)
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=11)

In [14]:
# oob_score : 일반화 정확도를 줄이기 위해 밖의 샘플 사용 여부
# oob : out of bag  / 36.8%정도는 뽑히지 않음 
# oob 점수는 **교차검증** 점수랑 비슷하게 나옴 
rf.oob_score_

0.8924379449682509

- OOB(out of bag) 샘플
    - 부트스트램 샘플에 포함되지 않아 훈련에 사용되지 않은 샘플
    - oob샘플을 검증 세트처럼 이용하여 훈련한 결정 트리를 평가할 수 있음 

In [17]:
# OOB 비율 계산 방법 

# 원본 데이터 세트의 크기
N = 1000

# 한 데이터 포인트가 한 번의 추출에서 선택되지 않을 확률
not_selected = 1 - 1/N 

# N번의 추출 동안 단 한번도 선택되지 않을 확률 
not_selected_N_times = not_selected**N

# OOB 데이터 비율 
not_selected_N_times

0.36769542477096373

# 엑스트라 트리

- 랜덤포레스트와 매우 유사함 
    - 기본적으로 100개의 의사결정나무를 훈련
    - 의사결정나무가 제공하는 대부분의 매개변수를 지원
    
- 랜덤포레스트와의 차이점
    - 부트스트랩 샘플을 사용하지 않음
        - 전체 훈련 세트를 사용함 
        
    - 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함
        - 하나의 의사결정나무에서 특성을 무작위로 분할한다면 성능이 낮아짐
        - 하지만 많은 트리를 앙상블하기 때문에 과대적합을 방지하고 검증 세트 점수를 높이는 효과가 나타남 

In [18]:
et = ExtraTreesClassifier(n_jobs = -1, random_state = 11)
scores = cross_validate(et, x_train, y_train, return_train_score = True, n_jobs = -1)

In [19]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9980276897524913 0.8870517139261123


- 예제에서는 독립변수가 많지 않아서 랜덤포레스트와 차이가 크지 않음
- 일반적으로 엑스트라트리가 무작위성이 더 크기 때문에 랜덤포레스트보다 더 많은 트리를 훈련해얗마
- 하지만 랜덤하게 노드를 분할하기 때문에 계산속도가 빠름 

In [20]:
# 엑스트라 트리도 특성중요도를 제공함
et.fit(x_train, y_train)
print(et.feature_importances_)

[0.20276578 0.51041115 0.28682308]


# 그레디언트 부스팅(gradient boosting)

- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 기법 
- 사이킷런의 GradientBoostingClassifier 는 기본적으로 깊이가 3인 결정 트리를 100개 사용함
    - 깊이가 얕은 결정 트리를 사용하여 과대적합을 방지할 수 있고 높은 일반화 성능을 기대할 수 있음 

In [21]:
gb = GradientBoostingClassifier(random_state = 11)
scores = cross_validate(gb, x_train, y_train, return_train_score = True, n_jobs = -1)

In [22]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.883490552338279 0.8639614644258533


- 그레디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 강함 

In [24]:
gb = GradientBoostingClassifier(random_state = 11, n_estimators = 500, learning_rate = 0.2)
scores = cross_validate(gb, x_train, y_train, return_train_score = True, n_jobs = -1)

In [25]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9393400192053069 0.8710801806470719


In [26]:
gb.fit(x_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, random_state=11)

In [27]:
gb.feature_importances_

array([0.17686589, 0.65930819, 0.16382592])